In [5]:
import pandas as pd
from collections import defaultdict
import os
from datetime import datetime
from scipy.stats import ttest_ind
from numpy import mean

In [6]:
#set constants

sources = ["Libero"]

timerange = ('2019-11-21',
             '2019-11-28')

folder = '../outputs/fb/summary'

file_users =  '../config/users.csv'


In [7]:
#load all impressions, filtering useful fields and replacing hased names with fb names


def absoluteFilePaths(directory):
    for dirpath, _, filenames in os.walk(directory):
        for f in filenames:
            if str(f)[0] == '.':
                pass
            else:
                yield os.path.abspath(os.path.join(dirpath, f))

def getImpressions(folder):
    impressions = pd.DataFrame()
    files = absoluteFilePaths(folder)
    tokens2names = pd.read_csv(file_users)
    for impressions_file in files:
        user_impressions = pd.read_csv(impressions_file)[['impressionTime', 'user', 'source']] #you can add fields of interest here
        name = tokens2names[tokens2names['token'] == impressions_file.split('/')[-1].split('.')[0]]['name'].values[0]
        user_impressions['user'] = name
        impressions = pd.concat([impressions, user_impressions])      
    return impressions

impressions = getImpressions(folder)

In [8]:
#filter out for desired time range for experiment


impressions_exp = impressions[(impressions['impressionTime'] >= timerange[0]) 
                        & (impressions['impressionTime'] <= timerange[1])]

In [11]:
#cut impressionTime to aggregate posts in the same timeline

impressions_exp['timeline'] = [impressionTime[:15]
                               for impressionTime in impressions_exp['impressionTime']]


/home/berra/dashboard/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [28]:
#count specific sources per timeline

def count_sources_by_timeline(impressions, sources):
    sources_timelines = {"source": [],
                     "timeline": [],
                     "user": [],
                     "count": []}
    for source in sources:
        for user_timeline, timeline_impressions in impressions_exp.groupby(['user','timeline']):
            sources_timelines["source"].append(source)
            sources_timelines["timeline"].append(user_timeline[1])
            sources_timelines["user"].append(user_timeline[0])
            sources_timelines["count"].append(timeline_impressions[timeline_impressions["source"] == source]["source"].count())

    return pd.DataFrame(data = sources_timelines)

sources_timelines = count_sources_by_timeline(impressions_exp, sources)

In [30]:
#exclude salvini+lega in order to harmonize shares across polarized and control

impressions_exp = impressions_exp[(impressions_exp["source"] != "Matteo Salvini") 
                        & (impressions_exp["source"] != "Lega - Salvini Premier")]

In [36]:
#normalize counts dividing for total impression per timeline

timeline_totalcount = impressions_exp.groupby(['user','timeline'])["source"].count()

sources_timelines["tot_count"] = [timeline_totalcount[timeline]
                              for timeline in sources_timelines["timeline"]] 

sources_timelines["share_count"] = (sources_timelines["count"] / sources_timelines["tot_count"])*100

sources_timelines

KeyError: '2019-11-21 07:0'